### Transfer Recommendation Model

- This model will be used so players can select a player that is currently in their team adn the model can give them an alternative based off of statistics for the season
- My plan for this is to use a K Nearest Neighbours (KNN) model.
- I am planning to use this model as it will give me the nearest neighbour to the selected player and this will emulate the functionality that I need with the most similar player for the user to select.

In [13]:
import pandas as pd
import numpy as np
import sklearn

# Load the data
df = pd.read_csv('24_25_prem_player_stats.csv')

print(df.head())

  Rk             Player   Nation    Pos        Squad     Age  Born  MP Starts  \
0  1         Max Aarons  eng ENG     DF  Bournemouth  25-107  2000   3      1   
1  2  Joshua Acheampong  eng ENG     DF      Chelsea  18-351  2006   4      2   
2  3        Tyler Adams   us USA     MF  Bournemouth  26-066  1999  23     16   
3  4   Tosin Adarabioyo  eng ENG     DF      Chelsea  27-209  1997  20     13   
4  5      Simon Adingra   ci CIV  FW,MF     Brighton  23-110  2002  24      8   

    Min  ... Ast.1 G+A.1 G-PK.1 G+A-PK  xG.1 xAG.1 xG+xAG npxG.1 npxG+xAG.1  \
0    86  ...  0.00  0.00   0.00   0.00  0.00  0.00   0.00   0.00       0.00   
1   170  ...  0.00  0.00   0.00   0.00  0.12  0.00   0.12   0.12       0.12   
2  1530  ...  0.18  0.18   0.00   0.18  0.09  0.05   0.14   0.09       0.14   
3  1229  ...  0.07  0.15   0.07   0.15  0.07  0.02   0.08   0.07       0.08   
4   826  ...  0.11  0.33   0.22   0.33  0.23  0.19   0.42   0.23       0.42   

   Matches  
0  Matches  
1  Matches  

In [17]:
# Ensure the first column is treated as a string and remove any leading/trailing spaces
df.iloc[:, 0] = df.iloc[:, 0].astype(str).str.strip()

# Filter out rows starting with 'Rk' (excluding the first row)
filtered_df = df.iloc[1:].copy()
filtered_df = filtered_df[~filtered_df.iloc[:, 0].str.startswith('Rk', na=False)]

# Re-add the first row (index 0) and reset index
df = pd.concat([df.iloc[[0]], filtered_df], ignore_index=True)

# Save the cleaned data
filtered_df.to_csv('24_25_prem_player_stats_clean.csv', index=False)

df = pd.read_csv('24_25_prem_player_stats_clean.csv')

/var/folders/jp/fbwfkpy50vs_7m6dlkzl468h0000gn/T/ipykernel_34338/1120704904.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0        2
1        3
2        4
3        5
4        6
      ... 
561    563
562    564
563    565
564    566
565    567
Name: Rk, Length: 566, dtype: object' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.iloc[:, 0] = df.iloc[:, 0].astype(str).str.strip()


This block here is just removing some lines that were unneeded as they were just repeats of the header line. They were there as the table was split up into smalller blocks on the html page

In [18]:
Defender = pd.read_csv('defending/24_25_defending.csv')

# Removing all of the players that arent defenders
Defender = Defender[Defender['Pos'].str.startswith("DF", na=False)]

Defender.to_csv('24_25_defending_clean.csv', index=False)
Defender = pd.read_csv('24_25_defending_clean.csv')

# Filling in the missing values with the mean of the column
Defender = Defender.fillna(Defender.mean(numeric_only=True))
df = df.fillna(df.mean(numeric_only=True))

# Assigning the X values based on the position of the player
Attacker_X = df[['xG.1', 'Gls.1', 'npxG+xAG.1', 'xAG.1']]
Midfielder_X = df[['PrgP', 'xAG.1', 'xG.1', 'PrgC']]
Defender_X = Defender[['Tkl%', 'Tkl+Int', 'Clr', 'Blocks', 'Sh']]

# Store player names separately
attacker_names = df['Player'].tolist()
midfielder_names = df['Player'].tolist()
defender_names = Defender['Player'].tolist()



Here I am reading in the defenders csv seperately as the statistics for the midfielders and attackers are all stored in my original dataset but my defender stats had to be gotten externally similarly to my previous model. The stats that I am going off of for this are the same ones that I used for my formula in my previous model, I am using thesde because they are based off of previous research that I have done from scouts and football statisticians.

My aim now is to train the KNN model with 1 neighbour to get a single player for them as a transfer recommendation. 

In [19]:
from sklearn.neighbors import NearestNeighbors

attacker_knn = NearestNeighbors(n_neighbors=1, metric='euclidean')
attacker_knn.fit(Attacker_X)

midfielder_knn = NearestNeighbors(n_neighbors=1, metric='euclidean')
midfielder_knn.fit(Midfielder_X)

defender_knn = NearestNeighbors(n_neighbors=1, metric='euclidean')
defender_knn.fit(Defender_X)

NearestNeighbors(metric='euclidean', n_neighbors=1)

Here I am just fitting the dataset to the model. I am using one neighbour to get the closest match to the sample data that is being inputted.

In [20]:
sample_attacker = np.array([0.35, 0.22, 0.48, 0.15])
sample_midfielder = np.array([5, 0.31, 0.21, 20])
sample_defender = np.array([35, 60, 18, 20, 10])

#  ChatGPT told me to format it like this as I was getting warnings when the sample wasn't being stored as a dataframe
sample_attacker_df = pd.DataFrame([sample_attacker], columns=Attacker_X.columns)
sample_midfielder_df = pd.DataFrame([sample_midfielder], columns=Midfielder_X.columns)
sample_defender_df = pd.DataFrame([sample_defender], columns=Defender_X.columns)

attacker_distance, attacker_index = attacker_knn.kneighbors(sample_attacker_df)
midfielder_distance, midfielder_index = midfielder_knn.kneighbors(sample_midfielder_df)
defender_distance, defender_index = defender_knn.kneighbors(sample_defender_df)

# Find the most similar player for each sample
similar_attacker = attacker_names[attacker_index[0][0]]
similar_midfielder = midfielder_names[midfielder_index[0][0]]
similar_defender = defender_names[defender_index[0][0]]

# Print the results
print(f"Most similar attacker: {similar_attacker}")
print(f"Most similar midfielder: {similar_midfielder}")
print(f"Most similar defender: {similar_defender}")

# Save the models
import joblib

joblib.dump(attacker_knn, 'attacker_transfer.pkl')
joblib.dump(midfielder_knn, 'midfielder_transfer.pkl')
joblib.dump(defender_knn, 'defender_transfer.pkl')

Most similar attacker: Anthony Gordon
Most similar midfielder: Luis Sinisterra
Most similar defender: Malo Gusto


['defender_transfer.pkl']

As you can see now it will recommend a player based off of the statistics inputted which I will change to the selected player in the future.